# Process lightning data from WWLN
This iPython notebook extracts lighning data from raw WWLN data files

Code by: Jasa Calogovic (Faculty of Geodesy, University of Zagreb)

Email: jcalogovic@geof.hr

In [ ]:
# Load required packages
import numpy as np
import datetime as dt
from datetime import timedelta
import pandas as pd
import os
from bokeh.plotting import Figure, show, output_notebook, vplot
from bokeh.charts import Bar
from bokeh.io import gridplot
from bokeh.models import HoverTool, Span, Range1d, LinearAxis
from bokeh.models.sources import ColumnDataSource

from stormstats.storm import WWLN

output_notebook()

## Various functions

In [ ]:
WWLN.Gen_date?

## Define parameters

In [ ]:
data_path='../data/WWLN/Jan2016/'

#start_time=
#end_time=
time_step=60   # in minutes
west_long_limit=0
east_long_limit=30
north_lat_limit=60
south_lat_limit=30
residuals_limit_us=2
minimum_no_stations=5

# Load data

In [ ]:
included_extenstions = ['loc']

listfiles = [fn for fn in os.listdir(data_path) if any(fn.endswith(ext) for ext in included_extenstions)]

In [ ]:
dates_data = []
for file in listfiles:
    dates_data.append(WWLN.Gen_date(file))

In [ ]:
file=data_path+listfiles[0]
test = pd.read_csv(file, index_col=0, parse_dates=True)

In [ ]:
test.head()